In [ ]:
# 查看显卡
!nvidia-smi

In [ ]:
#@title 克隆github仓库
!rm -rf /content/*

%cd /content/

!git lfs install
!git clone https://huggingface.co/spaces/YH-Official/SVC-Nahida

%cd /content/SVC-Nahida
!pip uninstall -y torchdata torchtext
!pip install --upgrade llvmlite
!pip install paddleaudio visualdl onnxruntime
!python -m pip install paddlepaddle-gpu==2.6.1.post117 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
!pip install --upgrade pip setuptools numpy numba
!pip install pyworld praat-parselmouth fairseq tensorboardX torchcrepe librosa==0.9.1
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117

# 推理

In [ ]:
#@title 上传.wav音频文件
import os
from google.colab import files

save_dir = "/content/SVC-Nahida/raw"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

uploaded = files.upload()
file_name = list(uploaded.keys())[0]
file_path = os.path.join(save_dir, file_name)
with open(file_path, "wb") as f:
    f.write(uploaded[file_name])
print(f"File saved at: {file_path}")


In [ ]:
#@title 合成音频（推理）修改
#@markdown 需要将音频上传到SVC-Nahida/raw 文件夹下, 然后设置模型路径、配置文件路径、合成的音频名称
model_path = "trained_models/纳西妲.pdparams" #@param {type:"string"}
config_path = "trained_models/config.json" #@param {type:"string"}
clean_names = "1.wav" # @param {type:"string"}
spk_list = "Nahida" #@param {type:"string"}
trans = "0"  #@param {type:"string"}
#@markdown **聚类音色泄漏控制**

#@markdown 0为完全不使用聚类，1为只使用聚类，通常设置0.5即可
cluster_infer_ratio = "0"  #@param {type:"string"}
slice_db = "-40"  #@param {type:"string"}
wav_format = "wav"  #@param {type:"string"}
wav_output = "/content/SVC-Nahida/results/" + clean_names + "_" + trans + "key" + "_" + spk_list + "." + wav_format

!python inference_main.py -m {model_path} -c {config_path} -n "{clean_names}" -t {trans} -s {spk_list}  -cr {cluster_infer_ratio} -sd {slice_db} -wf {wav_format}

#@markdown 如果你不想在此处下载，请取消勾选。
download_after_inference = True  #@param {type:"boolean"}

if download_after_inference:
  from google.colab import files
  files.download(wav_output)